In [2]:
import boto3
import sagemaker
import pandas as pd
from pyathena import connect

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


#### Auth with AWS

In [3]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

#### Convert csv to tsv and move to S3 

In [ ]:
s3_private_data_path = "s3://{}/w2-musicData/csv".format(bucket)
print(s3_private_data_path)

In [7]:
org_df = pd.read_csv('dataset.csv')
display(org_df.head())

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [9]:
org_df = org_df.drop(columns=['album_name'])
display(org_df.head())

,Unnamed: 0,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [11]:
org_df.to_csv('dataset_clean.csv', index=False)  

In [5]:
!aws s3 cp "dataset_clean.csv" $s3_private_data_path/

upload: ./dataset_clean.csv to s3://sagemaker-us-east-1-106006112223/w2-musicData/tsv/dataset_clean.csv


In [6]:
!aws s3 ls $s3_private_data_path/

2024-09-16 23:03:36   17618837 dataset_clean.csv
2024-09-15 20:52:01   20103228 dataset_tsv.csv


#### Create DB in Athena for queries

In [7]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
print(s3_staging_dir)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

s3://sagemaker-us-east-1-106006112223/athena/staging


In [22]:
database_name = "w2_music_db"
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
pd.read_sql(statement, conn)

/tmp/ipykernel_554/2702660478.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [8]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_253/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,w2_music_db


#### Create tables in DB and schemas

In [15]:
table_name_tsv = 'music_ds_tsv6'
database_name = "w2_music_db"
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         track_id string,
         artists string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit boolean,
         danceability int,
         energy int,
         key int,
         loudness int,
         mode int,
         speechiness float,
         acousticness float,
         instrumentalness float,
         liveness float,
         valence float,
         tempo float,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_tsv,s3_private_data_path
)

print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS w2_music_db.music_ds_tsv6(
         track_id string,
         artists string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit boolean,
         danceability int,
         energy int,
         key int,
         loudness int,
         mode int,
         speechiness float,
         acousticness float,
         instrumentalness float,
         liveness float,
         valence float,
         tempo float,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-106006112223/w2-musicData/tsv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


/tmp/ipykernel_253/1497903403.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [17]:
statement = "SHOW TABLES IN W2_MUSIC_DB"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_253/1294112312.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,music_ds_tsv6


In [21]:
# first test query
category = "Bad Bunny"

statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_tsv
)
print(statement)
df = pd.read_sql(statement, conn)
display(df.head(10))

# pull 200 rows from data for pandas way to query
statement = """SELECT * FROM {}.{}
    LIMIT 200""".format(
    database_name, table_name_tsv
)
print(statement)
full_df = pd.read_sql(statement, conn)
display(full_df.head(10))

SELECT * FROM w2_music_db.music_ds_tsv6 LIMIT 10


/tmp/ipykernel_253/98203777.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0\t5SuOikwiRyPMVoIQDJUgSV\tGen Hoshino\tComedy...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1\t4qPNDBW1i3p13qLCt0Ki3A\tBen Woodward\tGhost...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2\t1iJBSr7s7jYXzM8EGcbK5b\tIngrid Michaelson;Z...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3\t6lfxq3CG4xtTiEg7opyCyx\tKina Grannis\tCrazy...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4\t5vjLSffimiIP26QG5WcN2K\tChord Overstreet\tH...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,5\t01MVOl9KtVTNfFiBU9I7dc\tTyrone Wells\tDays ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,6\t6Vc5wAMmXdKIAM7WUoEb7N\tA Great Big World;C...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,7\t1EzrEOXmMH3G43AXT1y7pA\tJason Mraz\tWe Sing...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,8\t0IktbUcnAGrvD03AWnz3Q8\tJason Mraz;Colbie C...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,9\t7k9GuJYLp2AzqokyEdwEw2\tRoss Copperman\tHun...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


SELECT * FROM w2_music_db.music_ds_tsv6
    LIMIT 200


/tmp/ipykernel_253/98203777.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  full_df = pd.read_sql(statement, conn)


,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0\t5SuOikwiRyPMVoIQDJUgSV\tGen Hoshino\tComedy...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1\t4qPNDBW1i3p13qLCt0Ki3A\tBen Woodward\tGhost...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2\t1iJBSr7s7jYXzM8EGcbK5b\tIngrid Michaelson;Z...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3\t6lfxq3CG4xtTiEg7opyCyx\tKina Grannis\tCrazy...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4\t5vjLSffimiIP26QG5WcN2K\tChord Overstreet\tH...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,5\t01MVOl9KtVTNfFiBU9I7dc\tTyrone Wells\tDays ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,6\t6Vc5wAMmXdKIAM7WUoEb7N\tA Great Big World;C...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,7\t1EzrEOXmMH3G43AXT1y7pA\tJason Mraz\tWe Sing...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,8\t0IktbUcnAGrvD03AWnz3Q8\tJason Mraz;Colbie C...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,9\t7k9GuJYLp2AzqokyEdwEw2\tRoss Copperman\tHun...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Homework queries

#### 1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [12]:
statement = """SELECT artists, track_name, popularity FROM {}.{}
    WHERE popularity >= 99 LIMIT 10""".format(
    database_name, table_name_tsv
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(10)

SELECT artists, track_name, popularity FROM w2_music_db.music_ds_tsv4
    WHERE popularity >= 99 LIMIT 10


/tmp/ipykernel_253/944662429.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


DatabaseError: Execution failed on sql: SELECT artists, track_name, popularity FROM w2_music_db.music_ds_tsv4
    WHERE popularity >= 99 LIMIT 10
TYPE_MISMATCH: line 2:22: Cannot apply operator: varchar <= integer
unable to rollback

In [14]:
# pandas
# pd_df = full_df[full_df['popularity'] >= 99]
display(full_df.head(10))

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,None,73,230666,None,1.0,0,1.0,-6,0.0,0.0,0.0,0.0,0.0,1.0,87,4
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),None,55,149610,None,0.0,0,1.0,-17,1.0,0.0,1.0,0.0,0.0,0.0,77,4
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,None,57,210826,None,0.0,0,0.0,-9,1.0,0.0,0.0,0.0,0.0,0.0,76,4
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,None,71,201933,None,0.0,0,0.0,-18,1.0,0.0,1.0,0.0,0.0,0.0,181,3
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,None,82,198853,None,1.0,0,2.0,-9,1.0,0.0,0.0,0.0,0.0,0.0,119,4
5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,None,58,214240,None,1.0,0,6.0,-8,1.0,0.0,0.0,0.0,0.0,1.0,98,4
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,None,74,229400,None,0.0,0,2.0,-8,1.0,0.0,1.0,0.0,0.0,0.0,141,3
7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,None,80,242946,None,1.0,0,11.0,-9,1.0,0.0,1.0,0.0,0.0,1.0,150,4
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,None,74,189613,None,1.0,0,0.0,-8,1.0,0.0,0.0,0.0,0.0,1.0,130,4
9,9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,None,56,205594,None,0.0,0,1.0,-6,1.0,0.0,0.0,0.0,0.0,0.0,78,4


#### 2. List artists with an average popularity of 92


In [ ]:
# SELECT artists
# FROM w2_music_db
# GROUP BY artists
# HAVING AVG(popularity) = 92;


statement = """SELECT artists FROM {}.{}
    GROUP BY artists HAVING AVG(popularity) = 92""".format(
    database_name, table_name_tsv
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(10)

# pandas
# artists_avg_popularity = df.groupby('artists').filter(lambda x: x['popularity'].mean() == 92)
# artists_avg_popularity_list = artists_avg_popularity['artists'].unique()
# print(artists_avg_popularity_list)


#### 3. List the Top 10 most energetic genres
  

In [ ]:
# SELECT track_genre, AVG(energy) AS avg_energy
# FROM w2_music_db.tracks
# GROUP BY track_genre
# ORDER BY avg_energy DESC
# LIMIT 10;

statement = """SELECT track_genre, AVG(energy) AS avg_energy FROM {}.{}
    GROUP BY track_genre 
    ORDER BY avg_energy DESC LIMIT 10""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

# pandas
# top_energetic_genres = df.groupby('track_genre')['energy'].mean().sort_values(ascending=False).head(10)
# print(top_energetic_genres)


#### 4. How many tracks is Bad Bunny on?

In [ ]:
# SELECT COUNT(*) AS track_count
# FROM w2_music_db.tracks
# WHERE artists LIKE '%Bad Bunny%';

statement = """SELECT COUNT(*) AS track_count FROM {}.{}
    WHERE artists LIKE '%Bad Bunny%'""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

# bad_bunny_tracks_count = df[df['artists'].str.contains('Bad Bunny')].shape[0]
# print(bad_bunny_tracks_count)


#### 5. Show the top 10 genres in terms of popularity sorted by their most popular track

In [ ]:
# SELECT track_genre, MAX(popularity) AS max_popularity
# FROM w2_music_db.tracks
# GROUP BY track_genre
# ORDER BY max_popularity DESC
# LIMIT 10;

statement = """SELECT track_genre, MAX(popularity) AS max_popularity FROM {}.{}
    GROUP BY track_genre
    ORDER BY max_popularity DESC
    LIMIT 10""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

# pandas
# top_genres_by_popularity = df.groupby('track_genre')['popularity'].max().sort_values(ascending=False).head(10)
# print(top_genres_by_popularity)

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}